In [35]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Ign:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [36]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-04-13 22:06:16--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.2’

postgresql-42.2.16. 100%[===================>] 979.38K  5.71MB/s    in 0.2s    

2022-04-13 22:06:17 (5.71 MB/s) - ‘postgresql-42.2.16.jar.2’ saved [1002883/1002883]



In [37]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [40]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Office_Products_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   43081963|R18RVCKGH1SSI9|B001BM2MAC|     307809868|Scotch Cushion Wr...| Office Products|          5|            0|          0|   N|                Y|          Five Stars|      Great product.| 2015-08-31|
|         US|   10951564|R3L4L6LW1PUOFY|B00DZYEXPQ|      75004341|Dust-Off Compress...| Office Products|          5|    

In [41]:
df.count()

2642434

### Create DataFrames to match tables

In [66]:
from pyspark.sql.functions import to_date
from pyspark.sql.types import IntegerType
# Read in the Review dataset as a DataFrame


In [43]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   43622307|             1|
|   33972406|             1|
|     108460|             1|
|   43515569|             1|
|     132406|             1|
|   42560427|             1|
|    1673863|             2|
|   14552054|             1|
|   45632184|             1|
|   14703850|             2|
|   35768925|             3|
|   45640757|            28|
|   33194403|             1|
|   39383689|             1|
|   10645124|             1|
|   52512151|            10|
|   14646714|             1|
|   38659596|             1|
|   15634680|             1|
|   44178035|             1|
+-----------+--------------+
only showing top 20 rows



In [56]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id","product_title"]).drop_duplicates(["product_id"])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|074033039X|2014 Switched on ...|
|0741252465|2015 Office TriVi...|
|0760788367|Tuscany 2008 Wall...|
|1477010580|2015 Italy Wall C...|
|1562746693|National Parks 20...|
|1587799243|Disorders of the ...|
|1600224210|Proofreader’s Mar...|
|1604189207|Curriculum Word W...|
|1609310721|The New Yorker "S...|
|1609969596|Carson Dellosa Ma...|
|1619832445|2015 Marines, His...|
|1622264010|Orange Circle Stu...|
|1934026476|Crystal Springs B...|
|9981724793|Premium Screen Pr...|
|9988771126|Premium Screen Pr...|
|B00000J19L|Canon BC-20 Black...|
|B00000J3Q1|PalmOne V Slim Le...|
|B00000JD5Y|Canon BC-02 Black...|
|B00000JJ3S|Brother LC02Y Ink...|
|B00002EQE5|Fuga eDiary Elect...|
+----------+--------------------+
only showing top 20 rows



In [57]:
products_df.count()

313279

In [45]:
# Create the review_id_table DataFrame. 
#Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(["review_id","customer_id","product_id","product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R18RVCKGH1SSI9|   43081963|B001BM2MAC|     307809868| 2015-08-31|
|R3L4L6LW1PUOFY|   10951564|B00DZYEXPQ|      75004341| 2015-08-31|
|R2J8AWXWTDX2TF|   21143145|B00RTMUHDW|     529689027| 2015-08-31|
|R1PR37BR7G3M6A|   52782374|B00D7H8XB6|     868449945| 2015-08-31|
|R3BDDDZMZBZDPU|   24045652|B001XCWP34|      33521401| 2015-08-31|
| R8T6MO75ND212|   21751234|B004J2NBCO|     214932869| 2015-08-31|
|R2YWMQT2V11XYZ|    9109358|B00MOPAG8K|     863351797| 2015-08-31|
|R1V2HYL6OI9V39|    9967215|B003AHIK7U|     383470576| 2015-08-31|
|R3BLQBKUNXGFS4|   11234247|B006TKH2RO|     999128878| 2015-08-31|
|R17MOWJCAR9Y8Q|   12731488|B00W61M9K0|     622066861| 2015-08-31|
|R11EPG1GHOXMGB|   49861762|B00BXOGI3A|     688569009| 2015-08-31|
|R2797NKTEX5THN|    1541556|B0018RHWGE|     513803406| 2015-08

In [67]:
# Create the vine_table. DataFrame
# I encountered an error while inserting to table as start_rating was treated as string. So, added type conversion.
vine_df = df.select(["review_id", df["star_rating"].cast(IntegerType()),"helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R18RVCKGH1SSI9|          5|            0|          0|   N|                Y|
|R3L4L6LW1PUOFY|          5|            0|          1|   N|                Y|
|R2J8AWXWTDX2TF|          5|            0|          0|   N|                Y|
|R1PR37BR7G3M6A|          1|            2|          3|   N|                Y|
|R3BDDDZMZBZDPU|          4|            0|          0|   N|                Y|
| R8T6MO75ND212|          5|            0|          0|   N|                Y|
|R2YWMQT2V11XYZ|          5|            0|          0|   N|                N|
|R1V2HYL6OI9V39|          5|            6|          6|   N|                Y|
|R3BLQBKUNXGFS4|          5|            0|          0|   N|                Y|
|R17MOWJCAR9Y8Q|          5|            0|          0|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [47]:
# Configure settings for RDS
from getpass import getpass
password = getpass('Enter database password')
mode = "append"
jdbc_url="jdbc:postgresql://dataviz.c8ezqbfrrkjn.us-west-1.rds.amazonaws.com:5432/amazon_vine_reviews_db"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [48]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [58]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [59]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [68]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)